In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from openpyxl.chart import BarChart, Reference

from excel_manager import ExcelManager
from run_data_frame_calculations import RunDataFrameCalculationsForOneDataFrame


# Make a COPY before you run the script

In [2]:
path_input_excel = r"C:\Users\robbe\Downloads\python\python_plain_data.xlsx"

#make instance of excel manger
manager = ExcelManager(path_input_excel)

# load the workbook in
manager.load_workbook()

### You can see the sheet names

In [3]:
# check the sheet names
sheet_names = manager.get_sheet_names()
print(sheet_names)

['GT1.1', 'GT1.2', 'GT2.1', 'GT2.2']


#### input names of table if needed

In [4]:
# column_names = ['Sample ID', 'Parallel', 'Date', 'Time', 'P atm', 'P sample before gc', 'P sample after gc', 'CH4', 'CO2', 'O2', 'N2', 'Flush (1=yes; 0=no)', 'Comments', 'GC method', 'Weight', 'Date + Time', 'Day', 'Sum', 'CH4-corr', 'CO2-corr', 'O2-corr', 'N2-corr', 'Sum-corr', 'mg_bs', 'mCO2_b', 'mCH4_b', 'mO2_b', 'mN2_b', 'mCTot_b', 'mg_as', 'mCO2_a', 'mCH4_a', 'mO2_a', 'mN2_a', 'mCTot_a', 'mCTot_produced', 'O2 consumed', 'CO2 produced', 'O2 consumed_cum', 'CO2 produced_cum', 'mCTot_produced_cum', 'Cgas_DM_cum', 'PP CO2_b', 'CO2_aq [mol/m3]', 'CO2_aq [mol]', 'CO2_produced_aq', 'DIC_cum', 'Ctot_DM', 'Ratio O2/CO2']
#
# basic_column_names = ['Sample ID', 'Parallel', 'Date', 'Time', 'P atm', 'P sample before gc', 'P sample after gc', 'CH4', 'CO2', 'O2', 'N2', 'Flush (1=yes; 0=no)', 'Comments', 'GC method', 'Weight']

##### you can check if the sheets are in the instance of the excell manager class.

In [5]:
# check if you can see the data_frame
# full_sheet = manager.load_sheet_table(sheet_name="GT1.1", start_row=13)
#
# display(full_sheet)
# print(f' As you can see now the full sheet is shown')

### Filling th dictionary for the constants of the samples  in the instance of hte excel manager class

In [6]:
# fil in the dict with constants for the samples
sheet_names_samples = manager.get_sheet_names()
for sheet_name in sheet_names_samples:
    # loading the panda data frame for the sample
    constants_data_frame = manager.load_constants_as_data_frame(sheet_name=sheet_name, start_row=1, end_row=7, start_col=3)

    constants_data_class = manager.load_constants_as_data_class(sheet_name=sheet_name, start_row=1, end_row=7, start_col=3)

    # fill the dictionary
    manager.fill_dict_constants_data_frames(data_frame=constants_data_frame, sheets=[sheet_name])

    manager.fill_dict_constants_data_classes(data_class=constants_data_class, sheets=[sheet_name])

dict_constants = manager.get_dict_constants_data_frames()
dict_constants_data_classes = manager.get_dict_constants_data_classes()

#### running all the calculations for each data frame (sammple)

In [7]:
from nice_functions import NiceExcelFunction

# Now all the data frames are read in, calculated and returned in to the excell_file
sheet_names_samples = sheet_names[:]
for sheet_name in sheet_names_samples:
    # loading the data frame
    column_end = NiceExcelFunction.get_column_index(column_letter="O")
    data_frame = manager.load_sheet_table(sheet_name=sheet_name, start_row=13, end_column=column_end)

    # loading the data class with the constants for the sample
    constants_data_class = dict_constants_data_classes[sheet_name]

    # adding date and do the gas composition corrections.
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_data_frame_processor_calculations()
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_gas_composition_calculations()

    # calculations for moles before and after sampling
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_mol_gases_before_and_after_sampling(Rgas=constants_data_class.Rgas,
                                                                                                           exp_temperature=constants_data_class.expTemp,
                                                                                                           volume_headspace=constants_data_class.volume_headspace)

    # correcting the mg_bs for the first measurement
    data_frame.loc[0, "mg_as"] = data_frame.loc[0, "mg_bs"]

    # run mol gas composition
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_mol_gas_composition_calculation()
    # run moles produced
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_moles_produced()
    # run cumulative calculations carbon in gas phase
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_cumulative_production_in_the_gas_phase(molar_mass_carbon=constants_data_class.molar_mass_carbon,
                                                                                                              dry_mass_sample=constants_data_class.dry_mass_sample)

    # run calculation carbon in the aqueous phase
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_carbon_in_aqueous_phase(water_volume_in_liters=constants_data_class.water_volume,
                                                                                    dry_mass_sample=constants_data_class.dry_mass_sample)

    # run results interpretations
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_results_Interpretations()

    # print to see the process
    print(f"finished calculations {sheet_name}")

    # overwrite the old data frame with the new data frame which has calculations
    manager.fill_dict_panda_data_frames(data_frame=data_frame, sheets=[sheet_name])

    # print to see the process
    print(f"filled dictionary for {sheet_name}")

AttributeError: type object 'NiceExcelFunction' has no attribute 'get_column_index'

#### you can see the data frame below. Please fill the name of the tab below:

In [ ]:
sheet_tab_name = "GT1.1"


dict_data_frames = manager.get_dict_panda_data_frames()
display(dict_data_frames[sheet_tab_name])
# Set the display options
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Automatically adjust the column width to fit the contents

# OVERWRITING the excel file. Do you made a copy before?

In [ ]:
# sheet_names_samples = sheet_names[:]
# for sheet_name in sheet_names_samples:
#  # replace the table (overwrite) in the Excel file.
#     manager.replace_table_in_specific_sheet_with_data_frame(excel_file_path=path_input_excel,
#                                                             sheet_name=sheet_name,
#                                                             start_row=13,
#                                                             data_frame=data_frame,
#                                                             header=True
#                                                             )
#     print(f"replaced table in Excel for {sheet_name}")